In [3]:
import numpy as np
import pandas as pd

import re

In [4]:
f = open('inputs/day07_test.txt')
data = f.read()

In [23]:
def flatten(list):
    return [item for sublist in list for item in sublist]

class InstructionTree:
    
    def __init__(self, instructions):
        self.data = data
        self.node_names = list(pd.Series(flatten(re.findall('(?<= )[A-Z](?= )', data))).unique())
        self.nodes = {}
        for n in self.node_names:
            self.nodes[n] = Node(n)
        self.instructions = data.split('\n')
    
    def parse_line(self, line):
        names = re.findall('(?<= )[A-Z](?= )', line)    
        return names[0], names[1]
    
    def build_tree(self):
        for line in self.instructions:
            n0, n1 = self.parse_line(line)
            self.nodes[n0].add_child(self.nodes[n1])
            self.nodes[n1].add_parent(self.nodes[n0])
        
class Node:

    def __init__(self, name):
        self.name = name
        self.parents = []
        self.children = []
    
    def __repr__(self):
        return self.name
    
    def add_parent(self, parent):
        self.parents.append(parent)
        
    def add_child(self, child):
        self.children.append(child)

In [24]:
tree = InstructionTree(data)
tree.build_tree()

In [31]:
str([tree.nodes['C']])

'[C]'

In [28]:
tree.nodes['C'].children

[A, F]